In [174]:
import nltk

import numpy as np

import pandas as pd

import sklearn as sk
from sklearn import cross_validation
from sklearn import ensemble
from sklearn import feature_extraction
from sklearn import feature_selection
from sklearn import naive_bayes
from sklearn import neighbors
from sklearn import pipeline
from sklearn import svm


TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [18]:
### Load data

In [23]:
data = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
#help(data.head)
data.head(1)

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Dots,Number_of_Commas,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs,SentimentNumeric
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,...,1,0,0,0,4.4,0.7,2,0,0,2


In [27]:
#help(data)

In [76]:
neu = data_n[[
        "STATUS"
        , "cNEU"
        , "StringLength"
        , "Number_of_Words"
        , "Number_of_Dots"
        , "Number_of_Commas"
        , "Number_of_Semicolons"
        , "Number_of_Colons"
        , "Average_Word_Length"
        , "Lexical_Diversity"
        , "Number_of_FunctionalWords"
        , "Number_of_Pronouns"
        , "Number_of_PROPNAMEs"
        , "SentimentNumeric"
    ]]
neu.head(1)

,STATUS,cNEU,StringLength,Number_of_Words,Number_of_Dots,Number_of_Commas,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs,SentimentNumeric
0,likes the sound of thunder.,y,27,5,1,0,0,0,4.4,0.7,2,0,0,2


In [43]:
### Split Data in train and test sets
# http://scikit-learn.org/stable/modules/cross_validation.html

In [166]:

test_data = neu[["STATUS", "StringLength"]]
test_target = neu[["cNEU"]]

split = sk.cross_validation.train_test_split(test_data, test_target, train_size = 0.66, random_state= 6432119)
train_feat, test_feat, train_label, test_label = split

print("train_feat\n%s\n" %train_feat.head(1))
print("train_label\n%s\n" %train_label.head(1))
print("test_feat\n%s\n" %test_feat.head(1))
print("test_label\n%s\n" %test_label.head(1))



train_feat
                                                 STATUS  StringLength
8771  Merry Christmas & All The Love...  *PROPNAME* ...            79

train_label
     cNEU
8771    n

test_feat
                                                 STATUS  StringLength
7586  Cos all of the stars are fading away Just try ...            82

test_label
     cNEU
7586    n



In [46]:
### Create classifiers

In [82]:
## Support Vector Machines
# http://scikit-learn.org/stable/modules/svm.html

classifier_svc = ("support vector classifier", sk.svm.SVC(kernel='linear', C=1))
classifier_l_svc = ("linear svc", sk.svm.LinearSVC(random_state = 9974))


In [83]:
## Naive Bayes
# http://scikit-learn.org/stable/modules/naive_bayes.html
classifier_m_nb = ("Naive Bayes classifier for multinomial models", sk.naive_bayes.MultinomialNB())
classifier_b_nb = ("Naive Bayes classifier for multivariate Bernoulli models", sk.naive_bayes.BernoulliNB())

In [84]:
## Ensemble methods
# http://scikit-learn.org/stable/modules/ensemble.html
classifier_rfc = ("random forest classifier",
                  sk.ensemble.RandomForestClassifier(
                        max_depth=5
                        , n_estimators=10 
                        , max_features=1 
                        , n_jobs=-1
                        , random_state=32414343))
classifier_abc = ("Ada boost classifier", sk.ensemble.AdaBoostClassifier(random_state=67294))


In [85]:
## Neighbour methods
# http://scikit-learn.org/stable/modules/neighbors.html

classifier_knc = ("K nearest keighbours", sk.neighbors.KNeighborsClassifier())

In [93]:
## Pipelineing
# http://scikit-learn.org/stable/modules/pipeline.html
# http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html

pipeline = sk.pipeline.Pipeline([
        classifier_l_svc
    ])


pipeline

Pipeline(steps=[('linear svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=9974, tol=0.0001,
     verbose=0))])

In [165]:

classifier = classifier_svc[1]

#help(train_feat["StringLength"])

sk.cross_validation.cross_val_score(classifier, train_feat[["StringLength"]], train_label["cNEU"])


#help(classifier)

#trained_classifier = classifier.fit(train_feat, train_label)

#predicted = sk.cross_validation.cross_val_predict(trained_classifier, test_feat, test_label)

#print(predicted)

#predicted = sk.cross_validation.cross_val_predict(pipeline, train_feat, test_label, cv=10)
#metrics.accuracy_score(iris.target, predicted) 

array([ 0.62482822,  0.62494269,  0.62494269])

In [233]:
tfidf_vectorizer = sk.feature_extraction.text.TfidfVectorizer()

pipeline = sk.pipeline.Pipeline([('tfidf', tfidf_vectorizer),
                     ('chi2', sk.feature_selection.SelectKBest(sk.feature_selection.chi2, k=1000)),
                     ('nb', sk.naive_bayes.MultinomialNB())
                    ])

#pipeline.fit(train_feat[["STATUS"]], train_label[["cNEU"]])
data = train_feat["STATUS"]
target = train_label["cNEU"].apply(lambda x: x == 'y')

foo = tfidf_vectorizer.fit_transform(data[:1], target[:1])
print(foo)
#idf = foo.idf_

#print(len(idf))
#print(len(data))

result = sk.cross_validation.cross_val_score(pipeline, data, target)
print(result)

  (0, 6)	0.377964473009
  (0, 4)	0.377964473009
  (0, 2)	0.377964473009
  (0, 5)	0.377964473009
  (0, 0)	0.377964473009
  (0, 1)	0.377964473009
  (0, 3)	0.377964473009
[ 0.63078333  0.62998624  0.62906923]
